# 横向逻辑回归示例

这是一个使用Delta框架编写的横向联邦统计的任务示例。

数据是[spector dataset](https://www.statsmodels.org/stable/datasets/generated/spector.html)，关于个性化教学系统（PSI）计划有效性的实验数据。数据的格式是csv文件，包含4列：Grade, TCUE, PSI and GPA。任务是预测学生的GPA。

本示例可以直接在Deltaboard中执行并查看结果。<span style="color:#FF8F8F;font-weight:bold">在点击执行之前，需要修改一下个人的Deltaboard API的地址，具体请看下面第4节的说明。</span>

## 1. 引入需要的包

我们需要```delta-task```的包中，引入Delta框架的内容，包括```DeltaNode```节点，用于调用API发送任务，以及用于横向逻辑回归的```LogitTask```。


In [ ]:
import pandas
import delta.dataset
from delta import DeltaNode
from delta.statsmodel import LogitTask

## 2. 定义隐私计算任务

然后可以开始定义我们的横向统计任务了，用横向联邦统计的方式，在多节点上统计我们想要的数据。

在定义横向联邦统计任务时，有几部分内容是需要用户自己定义的：

* ***任务配置***: 我们需要在 ```super().__init__()``` 方法中对任务进行配置。 这些配置项包括任务名称（```name```），所需的最少客户端数（```min_clients```），最大客户端数（```max_clients```），等待超时时间（```wait_timeout```，用来控制一轮计算的超时时间），以及连接超时时间（```connection_timeout```，用来控制流程中每个阶段的超时时间）。

    另外，逻辑回归任务还可以在任务完成后，开启零知识证明阶段，用于验证最终结果的收敛性，以及各个节点计算过程中数据的一致性。如果要开启零知识证明，需要将`super().__init__()`中的`enable_verify`参数设置为True。同时，可以通过`verify_timeout`参数来控制零知识证明阶段的超时时间。目前，零知识证明阶段耗时较长，`verify_timeout`的默认值为300秒，如果在零知识证明阶段发生超时，建议适当加大`verify_timeout`。

    ***目前线上演示系统由于资源限制，暂不支持开启零知识证明阶段。请将`enable_verify`参数设置为False***

* ***数据集***: 我们需要在```dataset```方法中定义任务所需要的数据集。 该方法返回一个字典，键是数据集的名称，需要与execute方法的参数名对应；对应的值是```delta.dataset.DataFrame```实例， 其参数```dataset```代表所需数据集的名称。关于数据集格式的具体细节，请参考[这篇文章](https://docs.deltampc.com/network-deployment/prepare-data)。
* ***预处理***: 在预处理函数中，我们需要对数据集进行处理，最后返回x和y。 输入需要与```dataset```方法的返回值对应，即一个输入形参，对应```dataset```返回的字典中的一项。输出的x和y可以是`pandas.DataFrame`或`numpy.ndarray`，y必须是一个1维的向量，表示类别标签。
* ***选项配置***: 这个方法是可选的. 在`options`方法中，我们可以配置逻辑回归训练的一些参数。通用的参数包括 ```method```（逻辑回顾的训练方法，目前只有`newton`可选，即牛顿法）以及`maxiter`（训练的最大迭代次数）。还有一些牛顿法特有的参数, 包括`ord`（梯度范数的阶），`tol`（停止训练的容忍值）以及`ridge_factor`（对黑塞矩阵的脊回归系数）。上述所有的配置项，都有默认值。如果你没有特殊的需求，可以不实现这个方法。


In [ ]:
class SpectorLogitTask(LogitTask):
    def __init__(
        self,
    ) -> None:
        super().__init__(
            name="spector_logit",  # 任务名称，用于在Deltaboard中的展示
            min_clients=2,  # 算法所需的最少客户端数，至少为2
            max_clients=3,  # 算法所支持的最大客户端数，必须大雨等于min_clients
            wait_timeout=5,  # 等待超时时间，用来控制一轮计算的超时时间
            connection_timeout=5,  # 连接超时时间，用来控制流程中每个阶段的超时时间
            verify_timeout=500,  # 零知识证明步骤的超时时间
            enable_verify=False  # 是否开启零知识证明
        )

    def dataset(self):
        return {
            "data": delta.dataset.DataFrame("spector.csv"),
        }

    def preprocess(self, data: pandas.DataFrame):
        names = data.columns

        y_name = names[3]
        y = data[y_name].copy()  # type: ignore
        x = data.drop([y_name], axis=1)
        return x, y


## 3. 指定执行任务用的Delta Node的API

定义好了任务，我们就可以开始准备在Delta Node上执行任务了。

Delta Task框架可以直接调用Delta Node API发送任务到Delta Node开始执行，只要在任务执行时指定Delta Node的API地址即可。

Deltaboard提供了对于Delta Node的API的封装，为每个用户提供了一个独立的API地址，支持多人同时使用同一个Delta Node，并且能够在Deltaboard中管理自己提交的任务。
在这里，我们使用Deltaboard提供的API来执行任务。如果用户自己搭建了Delta Node，也可以直接使用Delta Node的API。

在左侧导航栏中进入“个人中心”，在Deltaboard API中，复制自己的API地址，并粘贴到下面的代码中：

In [ ]:
DELTA_NODE_API = "http://127.0.0.1:6704"

## 4. 执行隐私计算任务

接下来我们可以开始运行这个模型了：

In [ ]:
task = SpectorLogitTask().build()
delta_node = DeltaNode(DELTA_NODE_API)
delta_node.create_task(task)

## 5. 查看执行状态
点击执行后，可以从输出的日志看出，任务已经提交到了Delta Node的节点上。

接下来，可以从左侧的导航栏中，前往“任务列表”，找到刚刚提交的任务，点击进去查看具体的执行日志了。